In [ ]:
!pip install langchain mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.24
    Uninstalling langchain-core-0.3.24:
      Successfully uninstalled langchain-core-0.3.24
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.11
    Uninstalling langchain-0.3.11:
      Successfully uninstalled langchain-0.3.11


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.9 MB/s eta 0:00:

In [39]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.llms.base import LLM
from mistralai import Mistral, UserMessage
import argparse

api_key = "0vv9D9dhp384uuYpoAiTvBFTKPV0Tpjl"
model = "mistral-large-latest"
client = Mistral(api_key=api_key)

class MistralLLM(LLM):
    def _call(self, prompt: str, stop=None):
        messages = [UserMessage(content=prompt)]
        response = client.chat.complete(model=model, messages=messages)
        return response.choices[0].message.content.strip()

    @property
    def _identifying_params(self):
        return {"model": model}

    @property
    def _llm_type(self):
        return "mistral"

# Configuration for embeddings
stories_folder = Path("/content/langchain-assignment-dataset/stories")  # Adjust path as needed
vector_db_path = "/content/vector_db"
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [40]:
os.makedirs(vector_db_path, exist_ok=True)

# Function to compute embeddings
def compute_embeddings():
    if not stories_folder.exists():
        print(f"Directory not found: {stories_folder}")
        return

    documents = []
    for file_path in stories_folder.glob("*.txt"):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            documents.append(Document(page_content=content, metadata={"source": str(file_path)}))

    vector_store = Chroma.from_documents(documents, embedder, persist_directory=vector_db_path)
    vector_store.persist()
    print(f"Embeddings computed and saved to {vector_db_path}")

mistral_llm = MistralLLM()

# Function to retrieve character information
def get_character_info(character_name: str):
    # Load the vector database
    if not Path(vector_db_path).exists():
        print(f"Vector database not found: {vector_db_path}. Please run 'compute_embeddings' first.")
        return

    vector_store = Chroma(persist_directory=vector_db_path, embedding_function=embedder)
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    # Construct the query for detailed information
    query = f"Provide structured details about the character '{character_name}' in JSON format. Include name, storyTitle, summary, relations, and characterType."

    # Retrieve documents that potentially contain the character's name
    related_docs = retriever.get_relevant_documents(query)

    # Filter documents to include only those where the character's full name is mentioned
    matching_docs = [doc for doc in related_docs if character_name.lower() in doc.page_content.lower()]

    if not matching_docs:
        print(f"No documents containing character '{character_name}' were found in the embeddings.")
        return

    # If there are matching documents, proceed to extract and return the character's information
    context = "\n\n".join([doc.page_content for doc in matching_docs])
    prompt = f"Context:\n{context}\n\n{query}"
    response = mistral_llm(prompt)

    try:
        structured_response = json.loads(response)
        print(json.dumps(structured_response, indent=4))
    except json.JSONDecodeError:
        print(f"Could not parse response into JSON format. Response: {response}")

In [41]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create text input for character name (initially hidden)
character_name_input = widgets.Text(
    value='',
    placeholder='Type the character name here',
    description='Character:',
    disabled=False
)
character_name_input.layout.display = 'none'  # Hide the input initially

# Button to trigger embeddings computation
compute_button = widgets.Button(description="Compute Embeddings")

# Button to trigger character information retrieval (initially hidden)
get_info_button = widgets.Button(description="Get Character Info")
get_info_button.layout.display = 'none'  # Hide the button initially

# Output area for displaying results
output_area = widgets.Output()

def on_compute_clicked(b):
    with output_area:
        clear_output()
        print("Computing embeddings...")
        compute_embeddings()
        print("Embeddings computation done.")
        # Show character input and get info button after embeddings are computed
        character_name_input.layout.display = None  # Unhide the input
        get_info_button.layout.display = None  # Unhide the button

def on_get_info_clicked(b):
    with output_area:
        clear_output()
        character_name = character_name_input.value
        if character_name:
            print(f"Retrieving info for {character_name}...")
            get_character_info(character_name)
            print("Character info retrieval done.")
        else:
            print("Please enter a character name.")

# Attach the click event handlers to the buttons
compute_button.on_click(on_compute_clicked)
get_info_button.on_click(on_get_info_clicked)

# Display the widgets
display(compute_button, character_name_input, get_info_button, output_area)

Button(description='Compute Embeddings', style=ButtonStyle())

Text(value='', description='Character:', layout=Layout(display='none'), placeholder='Type the character name h…

Button(description='Get Character Info', layout=Layout(display='none'), style=ButtonStyle())

Output()